In [ ]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

asignacion_path = 'bases/asignacion/base_asignacion_20241002.xlsx'
df_asignacion = pd.read_excel(asignacion_path, sheet_name='BASE')
df_asignacion.shape

In [ ]:
def clean_columns(columns_list: list[str]) -> list[str]:
    return [column.strip().replace('  ', ' ').replace('  ', ' ').replace('.', '').replace(' ', '_').upper() for column in columns_list]

df_asignacion.columns = clean_columns(df_asignacion.columns)

if 'AGENCIA_CORRECTA' in df_asignacion.columns:
    df_asignacion.rename(columns={'AGENCIA_CORRECTA': 'AGENCIA'}, inplace=True)

cols_asignacion = ['CODIGO', 'CONT_18', 'NOMBRE', 'TIPO_CARTERA', 'TIPO_FONDO', 'CLAVE', 'AGENCIA']
df_asignacion = df_asignacion[cols_asignacion]

df_asignacion['CONT_18'] = df_asignacion['CONT_18'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)
df_asignacion['CODIGO'] = df_asignacion['CODIGO'].astype('Int64').astype(str).str.zfill(8)

df_asignacion['FLAG'] = 0
conteo_cc = df_asignacion['CODIGO'].value_counts()
df_asignacion['FLAG'] = df_asignacion['CODIGO'].map(conteo_cc)

print(df_asignacion.shape)
df_asignacion.head()

In [13]:
df_asignacion.to_excel('bases/asignacion/base_asignacion.xlsx', index=False)